In [22]:
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import  LabelEncoder
from sklearn.tree import DecisionTreeRegressor


# easy way of accessing A_6, A_7, ... A_N columns
def column_list(letter, start, end):
    return ["%s%d" % (letter, i) for i in range(start, end)]


# convert strings to int type even if it's a float
# replace by median or mean?
def convert_stat(x, new_value=0):
    if not isinstance(x, int):

        if not isinstance(x, float) and '.' not in x:
            return new_value if x == "TRANSFER" else int(x)
        else:
            return new_value if x == "TRANSFER" else int(float(x))

    else:
        return x


class SumTransformer(BaseEstimator):

    # set new_value to None if Pipeline contains SimpleImputer
    # this is for absences and tardies
    def __init__(self, new_value=0):
        self.new_value = new_value

    def fit(self, x, y=None):
        return self

    def transform(self, df):
        for i in ["A", "T"]:
            for j in column_list(i, 6, 13):
                df[j] = df[j].apply(convert_stat, self.new_value)

        df['AbsentSum'] = df[column_list('A', 6, 13)].sum(axis=1)
        df['TardySum'] = df[column_list('T', 6, 13)].sum(axis=1)

        # for different time periods
        df['AbsencesSum_MS'] = df[column_list('A', 6, 9)].sum(axis=1)
        df['AbsencesSum_HS'] = df[column_list('A', 9, 13)].sum(axis=1)

        df['TardiesSum_MS'] = df[column_list('T', 6, 9)].sum(axis=1)
        df['TardiesSum_HS'] = df[column_list('T', 9, 13)].sum(axis=1)

        return df


pre_process = ColumnTransformer(remainder='passthrough',
                                transformers=[('categories', LabelEncoder(), ["Gender", "IEP/Specialized"])])

model_pipeline = Pipeline(steps=[('number_fix', SumTransformer()),
                                 ('pre_process', pre_process),
                                 ('Decision Tree', DecisionTreeRegressor(max_leaf_nodes=12))
                                 ])

from sklearn.model_selection import train_test_split, cross_val_score
features = ["A6", "A7", "A8", "A9", "Gender", "IEP/Specialized"]
student_data = pd.read_csv("../data/High School East Student Data - Sheet1.csv")
student_data["AbsencesSum_HS"] = 0

scores = -1 * cross_val_score(model_pipeline,
            student_data[features],
            student_data["AbsencesSum_HS"],
            cv=4,
            scoring="neg_mean_absolute_error")

print(scores)


[nan nan nan nan]


<ipython-input-22-80d7cb55eec5>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[j] = df[j].apply(convert_stat, self.new_value)
/Users/Turtle/.conda/envs/2020 Research/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Turtle/.conda/envs/2020 Research/lib/python3.8/site-packages/pandas/core/indexes/base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pand

> /Users/Turtle/PycharmProjects/2020 Research/model/pandas/_libs/hashtable_class_helper.pxi(1627)pandas._libs.hashtable.PyObjectHashTable.get_item()

